# Use GPIO library to control an output device: DC Motor
In this exercise, we try to use GPIO library to control another output device: DC Motors.

## Turn off bluetooth_service
Before we start, ensure you turn off bluetooth_control service on the RPi car. The service is there for your Android app to communicate actions to remote control the car. We do not need that now. Leaving it on would interfere with the GPIO exercises here. Open a terminal from Jupyter and type the command below after the dollar sign.
```bash
$ sudo systemctl stop bluetooth_control
```

## Code cells
You will find plenty of code cells that contain Python code to execute on the RPi car. A Jupyter server runs on the RPi car and receives the code from these cells in browser for the car to execute it locally on the RPi.

Follow each of the code cells below, some cells contain complete code you can run, while others contain incomplete code. You can always modify or add to the code in order to experiment.

Simply press "shift enter" together to execute the code.

## Comments
Python uses \# to provide comments that can illustrate what a programmer intends the code to do. You should read comments to help you understand the code and write comments to help readers and yourself make sense of your own code.

# Exercises
## Caution
<font color=red>For any coding exercises that you perform here, be sure that your RPi car sit on a safe surface so it would not fall off and break into pieces.</font>

## Make motors run forward
With GPIO library, making motors run is quite similar to making LED light up. You just need to figure out which pins to set in what order. The truth table is printed here:

| Enable | Forward | Backward | Result  |
| ------ |---------|----------|---------|
| Low    | -       | -        |Not spinning - not enabled |
| High   | Low     | Low      |Not spinning - both forward and backward are off |
| High   | High    | Low      |Spinning forward |
| High   | High    | Low      |Spinning backward |
| High   | High    | High     |Not spinning - both forward and backward are on |

With Yahboom's car kit via the motor driver chip, the forward/backward/enable GPIO pins for the left motors and right motors are configured this way:

| Broadcom pin# | Enable | Forward | Backward |
|-|--------|---------|----------|
|Left motors|16|20|21|
|Right motors|13|19|26|

The code cell below has:
* setup_pins() function that sets up controller pins as output for one side of motors.
* run_forward() function that sends signal output for motors to run forward

In [23]:
import RPi.GPIO as GPIO

# Yahboom's car kit pin setup for the four motors:
# two motors on th left and two motors on the right
PIN_MOTOR_LEFT_ENABLE = 16
PIN_MOTOR_LEFT_FORWARD = 20
PIN_MOTOR_LEFT_BACKWARD = 21

PIN_MOTOR_RIGHT_ENABLE = 13
PIN_MOTOR_RIGHT_FORWARD = 19
PIN_MOTOR_RIGHT_BACKWARD = 26

def setup_pins(pin_forward, pin_backward, pin_enable):
    """
    Sets up a set of controller pins (enable/forward/backward) 
    as output for one side of motors
    """
    GPIO.setmode(GPIO.BCM)
    GPIO.setwarnings(False)
    GPIO.setup(pin_enable, GPIO.OUT, initial=GPIO.LOW)
    GPIO.setup(pin_forward, GPIO.OUT, initial=GPIO.LOW)
    GPIO.setup(pin_backward, GPIO.OUT, initial=GPIO.LOW)

def run_forward(pin_forward, pin_backward, pin_enable):
    """
    * Sets up pins as output
    * Sends controller signals to prepare motors to run forward
    * Sends enable signal to run motor
    Inputs
    ------
    * pin_forward, pin_backward, pin_enable - the broadcom pin numbers to set for controller output
    """
    # Refer to the truth table.
    # Send controller signals to prepare motors to run forward
    GPIO.output(pin_forward, GPIO.HIGH)
    GPIO.output(pin_backward, GPIO.LOW)
    # Pause the program to wait for user input to start
    input("Press return to start")
    # Send enable signal to run motor
    GPIO.output(pin_enable, GPIO.HIGH)
    # Pause the program to wait for user input to stop while motor is running
    input("Press return to stop")
    # Clean up all pins used in this program
    GPIO.cleanup()

# Run the right motors forward
setup_pins(PIN_MOTOR_RIGHT_FORWARD, PIN_MOTOR_RIGHT_BACKWARD, PIN_MOTOR_RIGHT_ENABLE)
run_forward(PIN_MOTOR_RIGHT_FORWARD, PIN_MOTOR_RIGHT_BACKWARD, PIN_MOTOR_RIGHT_ENABLE)

Press return to start
Press return to stop


## Make motors run backward
Now you have seen the truth table and the sample code, can you define a function that makes the motors run backward? Once you have defined the function, can you make the left-hand-side motors run backward? Use the cell below to complete this task.

Note: if your RPi has been restarted, and you come back to this code cell directly, you must run the code cell above to make sure necessary libraries have been imported and necessary variables have been defined, because your cell below does not do the importing and defining.

In [14]:
# TODO: define this function
def run_backward(your_input_parameter_list_here):
    pass

# TODO: use the function you just defined to run left motors backward
run_backward(your_input_value_for_left_motors)

## Use PVM to control speed
You must have noticed that when you output GPIO.HIGH to the enable pin, the motors spin at full speed. There does not seem to a speed throttling mechanism with this simple signal. 

Recall that we discussed PVM with LED? Very similar to how we use duty cycle in PVM to control the brightness of LED, we can use duty cycle to control the speed of motors.

In the code cell below, rather than using GPIO.output(HIGH) for the enable pin, we will use PVM and duty cycle to control the percentage of time during which motor runs, to achieve the effect of speed control. 

Experiment with different duty cycle to see the effect of different speed setting.

Also, note that [conventionally](https://electronics.stackexchange.com/questions/242293/is-there-an-ideal-pwm-frequency-for-dc-brush-motors) we use a high frequency (>= 2000Hz) to set DC motor PVM frequency. 

In [13]:
import RPi.GPIO as GPIO

PIN_MOTOR_LEFT_ENABLE = 16
PIN_MOTOR_LEFT_FORWARD = 20
PIN_MOTOR_LEFT_BACKWARD = 21

PIN_MOTOR_RIGHT_ENABLE = 13
PIN_MOTOR_RIGHT_FORWARD = 19
PIN_MOTOR_RIGHT_BACKWARD = 26

def setup_pins_with_enable_pvm(pin_forward, pin_backward, pin_enable):
    """
    This function is identical to setup_pins() defined above
    It just sets all three pins as output
    """
    GPIO.setmode(GPIO.BCM)
    GPIO.setwarnings(False)
    GPIO.setup(pin_enable, GPIO.OUT, initial=GPIO.LOW)
    GPIO.setup(pin_forward, GPIO.OUT, initial=GPIO.LOW)
    GPIO.setup(pin_backward, GPIO.OUT, initial=GPIO.LOW)

def run_forward_with_enable_pvm(pin_forward, pin_backward, pin_enable, duty_cycle):
    """
    * Sets up all 3 controller pins as output
    * Sends controller signals to prepare motors to run forward
    * Sets up PVM for enable pin.
    Inputs
    ------
    * pin_forward, pin_backward, pin_enable - the broadcom pin numbers to set for controller output
    * duty_cycle - duty_cycle on pin_enable to control the speed of motors, a number between [0,100]
    """
    setup_pins_with_enable_pvm(pin_forward, pin_backward, pin_enable)
    # Send controller signals to prepare motors to run forward
    GPIO.output(pin_forward, GPIO.HIGH)
    GPIO.output(pin_backward, GPIO.LOW)
    # set PWM for enable pin with frequency 2000 Hz
    pwm = GPIO.PWM(pin_enable, 2000)
    # Pause the program to wait for user input to start
    input("Press return to start at speed = {s}".format(s=duty_cycle) )
    # Start the pwm pulsing with specified duty_cycle
    pwm.start(duty_cycle)
    # Pause the program to wait for user input to stop while motor is running
    input("Press return to stop")
    # Clean up all pins used in this program
    pwm.stop()
    GPIO.cleanup()

# Run the right motors forward with duty cycle (like setting speed)
speed_dc = 20 # number between [0, 100]
setup_pins_with_enable_pvm(PIN_MOTOR_RIGHT_FORWARD, PIN_MOTOR_RIGHT_BACKWARD, PIN_MOTOR_RIGHT_ENABLE)
run_forward_with_enable_pvm(PIN_MOTOR_RIGHT_FORWARD, PIN_MOTOR_RIGHT_BACKWARD, PIN_MOTOR_RIGHT_ENABLE, speed_dc)

Press return to start at speed = 20
Press return to stop


## Turn the car
Since we don't have a front axle that can turn, we can't steer the car the way we do a real vehicle. Instead, we can rely on the difference in speeds between motors on two sides to achieve effective turning:
* turning speed
* turning radius

In the code cell below, we re-organize the code we have written so far so that it becomes easier to write a turning car.

1. Try to understand all the functions in this code cell
2. Then experiment in the next code cell below to write your own functions and moving routines.

In [1]:
import RPi.GPIO as GPIO
import time

PIN_MOTOR_LEFT_ENABLE = 16
PIN_MOTOR_LEFT_FORWARD = 20
PIN_MOTOR_LEFT_BACKWARD = 21

PIN_MOTOR_RIGHT_ENABLE = 13
PIN_MOTOR_RIGHT_FORWARD = 19
PIN_MOTOR_RIGHT_BACKWARD = 26

PWM_LEFT_SPEED = None
PWM_RIGHT_SPEED = None

def setup_all_6_pins():
    """
    * Set up all 6 pins for output, 3 for the left, 3 for the right
    * Set up PWM's for left motor speed and right motor speed controls
    """
    GPIO.setmode(GPIO.BCM)
    GPIO.setwarnings(False)
    GPIO.setup(PIN_MOTOR_LEFT_ENABLE, GPIO.OUT, initial=GPIO.LOW)
    GPIO.setup(PIN_MOTOR_LEFT_FORWARD, GPIO.OUT, initial=GPIO.LOW)
    GPIO.setup(PIN_MOTOR_LEFT_BACKWARD, GPIO.OUT, initial=GPIO.LOW)
    GPIO.setup(PIN_MOTOR_RIGHT_ENABLE, GPIO.OUT, initial=GPIO.LOW)
    GPIO.setup(PIN_MOTOR_RIGHT_FORWARD, GPIO.OUT, initial=GPIO.LOW)
    GPIO.setup(PIN_MOTOR_RIGHT_BACKWARD, GPIO.OUT, initial=GPIO.LOW)
    
    global PWM_LEFT_SPEED
    global PWM_RIGHT_SPEED

    PWM_LEFT_SPEED  = GPIO.PWM(PIN_MOTOR_LEFT_ENABLE, 2000)
    PWM_LEFT_SPEED.start(0)
    PWM_RIGHT_SPEED = GPIO.PWM(PIN_MOTOR_RIGHT_ENABLE, 2000)
    PWM_RIGHT_SPEED.start(0)
    
def left_motors_run(speed_dc, time_in_seconds=0):
    """
    Run the left motors at speed_dc for time_in_seconds
    Inputs
    ------
    speed_dc - a number between [-100, +100]
               negative numbers run the left motors backward
               positive numbers run the left motors forward
    time_in_seconds - run the left motors at specified speed
                      for this duration before motors stop spinning
                    - default time length is 0, i.e. motors will
                      not stop spinning
    """
    if speed_dc > 0:
        GPIO.output(PIN_MOTOR_LEFT_FORWARD, GPIO.HIGH)
        GPIO.output(PIN_MOTOR_LEFT_BACKWARD, GPIO.LOW)
    else:
        GPIO.output(PIN_MOTOR_LEFT_FORWARD, GPIO.LOW)
        GPIO.output(PIN_MOTOR_LEFT_BACKWARD, GPIO.HIGH)
    global PWM_LEFT_SPEED
    PWM_LEFT_SPEED.ChangeDutyCycle(abs(speed_dc))
    if time_in_seconds > 0:
        time.sleep(time_in_seconds)
        PWM_LEFT_SPEED.ChangeDutyCycle(0)

def right_motors_run(speed_dc, time_in_seconds=0):
    """
    Run the right motors at speed_dc for time_in_seconds
    Inputs
    ------
    speed_dc - a number between [-100, +100]
               negative numbers run the right motors backward
               positive numbers run the right motors forward
    time_in_seconds - run the right motors at specified speed
                      for this duration before motors stop spinning
                    - default time length is 0, i.e. motors will
                      not stop spinning
    """
    if speed_dc > 0:
        GPIO.output(PIN_MOTOR_RIGHT_FORWARD, GPIO.HIGH)
        GPIO.output(PIN_MOTOR_RIGHT_BACKWARD, GPIO.LOW)
    else:
        GPIO.output(PIN_MOTOR_RIGHT_FORWARD, GPIO.LOW)
        GPIO.output(PIN_MOTOR_RIGHT_BACKWARD, GPIO.HIGH)
    global PWM_RIGHT_SPEED
    PWM_RIGHT_SPEED.ChangeDutyCycle(abs(speed_dc))
    if time_in_seconds > 0:
        time.sleep(time_in_seconds)
        PWM_RIGHT_SPEED.ChangeDutyCycle(0)

def run(left_speed, right_speed, time_in_seconds=0):
    """
    Run all the motors at specified speeds for the left and the right 
    for the duration of time_in_seconds
    
    Inputs
    ------
    left_speed - a number between [-100, +100]
               negative numbers run the left motors backward
               positive numbers run the left motors forward
    right_speed - a number between [-100, +100]
               negative numbers run the right motors backward
               positive numbers run the right motors forward
               
    time_in_seconds - run the motors at specified speeds
                      for this duration before motors stop spinning
                    - default time length is 0, i.e. motors will
                      not stop spinning
    """
    left_motors_run(left_speed)
    right_motors_run(right_speed)
    if time_in_seconds > 0:
        time.sleep(time_in_seconds)
        PWM_LEFT_SPEED.ChangeDutyCycle(0)
        PWM_RIGHT_SPEED.ChangeDutyCycle(0)

def stop():
    """
    Properly stop all the motors and reset all pins
    """
    PWM_LEFT_SPEED.stop()
    PWM_RIGHT_SPEED.stop()
    GPIO.cleanup()

# Main program starts below
# You should use try-finally structure to ensure proper stop and cleanup
try:
    # Must first set up the pins for controls
    setup_all_6_pins()
    print("Test left motors")
    left_motors_run(-20, 2)
    time.sleep(1)
    
    print("Test right motors")
    right_motors_run(20, 2)
    time.sleep(1)
    
    print("Test motors on both sides run forward for 2 seconds")
    run(40, 40, 2)
    time.sleep(1)
    
    print("Test motors on both sides run backward for 2 seconds")
    run(-40, -40, 2)
    time.sleep(1)
    
    print("Test motors on both sides turn right (40,10) for 2 seconds")
    run(40, 10, 2)
    time.sleep(1)
    
    print("Test motors on both sides turn right (40, 0) for 2 seconds")
    run(40, 0, 2)
    time.sleep(1)
    
    print("Test motors on both sides turn right (40, -10) for 2 seconds")
    run(40, -10, 2) 
    time.sleep(1)
finally:
    # The following is the code to ensure proper car stop and cleanup of pins
    stop()

Test left motors
Test right motors
Test motors on both sides run forward for 2 seconds
Test motors on both sides run backward for 2 seconds
Test motors on both sides turn right (40,10) for 2 seconds
Test motors on both sides turn right (40, 0) for 2 seconds
Test motors on both sides turn right (40, -10) for 2 seconds


### Challenge
Write your own run functions calling the run() functions defined above with different speed and time parameters.

Be sure that you use try-finally structure to ensure proper cleanup of pins after the routine is finished.

Experiment and see if you can make the car run in a square.

In [ ]:
# TODO Your run functions
def your_own_run_1():
    """
    Define your own run function here using the run() function defined above using 
    * combination of left_speed, right_speed to control moving and turning 
    * time_in_second to control duration of moving and turning
    """
    pass
def your_own_run_2():
    """
    Define your own run function here using the run() function defined above using 
    * combination of left_speed, right_speed to control moving and turning 
    * time_in_second to control duration of moving and turning
    """
    pass

# Main program starts below
# You should use try-finally structure to ensure proper stop and cleanup
try:
    # Must first set up the pins for controls
    setup_all_6_pins()
    
    # TODO Your run routine starts here
    
finally:
    # The following is the code to ensure proper car stop and cleanup of pins
    stop()